# HW09: Transformers

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [71]:
import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label)
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

,label,title,lead,text
6170,sport,Muller upsets Agassi in semis,Luxembourg #39;s Gilles Muller is ranked so lo...,Muller upsets Agassi in semis Luxembourg #39;s...
13491,sport,PGA Changes Dates of Grand Slam of Golf (AP),"AP - The Grand Slam of Golf, which brings toge...",PGA Changes Dates of Grand Slam of Golf (AP) A...
102878,world,Typhoon flooding kills 25 in Vietnam,Torrential rains hit the region this week and ...,Typhoon flooding kills 25 in Vietnam Torrentia...
117468,business,Baseball Wrangling Makes Some Uneasy,Tuesday night's D.C. Council vote on baseball ...,Baseball Wrangling Makes Some Uneasy Tuesday n...
105047,world,U.S. President Bush aims to avoid missteps on ...,Canadian Press - WASHINGTON (AP) - President G...,U.S. President Bush aims to avoid missteps on ...


## Hugginface Transformers

In [72]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
import tensorflow as tf

config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
config.num_labels = 4
transformer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

In [73]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text
input_ids = tf.keras.layers.Input(shape=(8,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(8,), name='masked_token', dtype='int32')
X = transformer_model(input_ids, input_masks_ids)
model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

In [74]:
##TODO print the summary of the model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 8)]          0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 8)]          0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla ((None, 4),)         66956548    input_token[0][0]                
                                                                 masked_token[0][0]               
Total params: 66,956,548
Trainable params: 66,956,548
Non-trainable params: 0
__________________________________________________________________________________________________


In [75]:
##TODO compile the model

model.compile(loss='sparse_categorical_crossentropy', # cost function
              optimizer='adam', # use adam as the optimizer
              metrics=['accuracy']) # compute accuracy, for scoring

**Hint:** All the vectorized pieces of text must have the same length (which will be equal to the input size). You have two options to ensure this:

1. Set the maximum length equal to the length of the shortest vectorized text
2. Choose the maximum length and then exclude all the data points that have vectors shorter than that length

**Hint:** Tensorflow requires your labels to be integers, not strings

In [76]:
##TODO split the sample into a training and a test set
from sklearn.model_selection import train_test_split
df.label = df.label.replace({'world': 0, 'sport':1, 'business': 2, 'sci/tech':3})

X_train, X_test, y_train, y_test = train_test_split(df['lead'].tolist(), df['label'].tolist(), test_size=.2)
text_lengths = sorted({len(x) for x in df.lead})
print(text_lengths)

max_text_length = text_lengths[0]

[32, 39, 42, 45, 48, 52, 53, 54, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260

In [77]:
##TODO prepare the dataset for tensorflow.
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
X_train_tf = [tokenizer(x, return_tensors="tf", padding=True, truncation=True, max_length=8) for x in X_train]

In [78]:
input_ids, input_masks = [x["input_ids"][0].numpy() for x in X_train_tf], [x["attention_mask"][0].numpy() for x in X_train_tf]

In [79]:
print(sorted({len(x) for x in input_ids}))
print(sorted({len(x) for x in input_masks}))

[8]
[8]


In [80]:
dataset = tf.data.Dataset.from_tensor_slices(({'input_token': input_ids, 'masked_token': input_masks}, y_train)).batch(1)

In [81]:
##TODO fit the model and print the obtained accuracy
model_info = model.fit(dataset,epochs=1)

 226/8000 [..............................] - ETA: 1:24:25 - loss: 4.8865 - accuracy: 0.3177      

KeyboardInterrupt: 

## Generating Text

In [85]:
#!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [89]:
from nltk import word_tokenize
for label in range(4):
    print("*" * 30 + label_map[label +1] + "*" * 30)
    text = df[df.label == label].iloc[0].lead
    print('True text:\n ', text)

    framer_text = ' '.join(word_tokenize(text)[:5])
    input_ids = tokenizer.encode(framer_text, return_tensors="pt")

    print('Framing text:\n', framer_text)

    sample_output = model.generate(input_ids, max_length=50)
    generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    print('Generated Text:\n', generated_text)
    print('\n')

##TODO Pick one snippet for each label and generate some starting from the first 4-5 words
## pick the generating model that looks best to you (and explain why) and set the length of each generated document to 50


******************************world******************************
True text:
  Torrential rains hit the region this week and floods submerged more than 10,000 houses and brought traffic to a standstill. A disaster management official from the coastal province of Quang Ngai says a red 
Framing text:
 Torrential rains hit the region
Generated Text:
 Torrential rains hit the region on September 28, killing at least nine people and leaving dozens of people missing AFP/Getty 30/50 5 September 2018 US Capitol Police arrest a protestor as Judge Brett Kavanaugh testifies during the second day of his US


******************************sport******************************
True text:
  Luxembourg #39;s Gilles Muller is ranked so low he has to qualify next week just to see if he can even get into the US Open in less than two weeks.
Framing text:
 Luxembourg # 39 ; s
Generated Text:
 Luxembourg # 39 ; s.v. "to be a man of the people." — Der. Luxembourg, Luxembourg, Luxembourg, Luxembourg, Luxembourg